* Karpathy video: https://www.youtube.com/watch?v=kCc8FmEb1nY
* Karpathy repo: https://github.com/karpathy/nanoGPT
* Colab for video: https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing

# Setup:

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np 

!wget -N https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


--2023-11-25 19:07:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

Last-modified header missing -- time-stamps turned off.
2023-11-25 19:07:02 (18.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
use_wandb = True
if use_wandb:
    import wandb
    wandb.login()

wandb: Currently logged in as: drscotthawley. Use `wandb login --relogin` to force relogin


In [3]:
# Karpathy's # hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

config = {
  'learning_rate': learning_rate,
  'batch_size': batch_size,
  'block_size': block_size,
  'n_embd': n_embd,
  'n_head': n_head,
  'n_layer': n_layer,
  'dropout': dropout
}

In [4]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

print('device =',device) 

device = cuda:1


# Dataset creation

Codebook creation

In [5]:
chars = list(text)
# data here.
#raw_data = torch.tensor([ord(c) for c in chars],dtype=torch.int32).unsqueeze(1) # sadly torch has no char dtype
raw_data = np.array(list(text))[:,np.newaxis] # but numpy does
print("raw_data.shape =",raw_data.shape)
n_codebooks = raw_data.shape[-1]
print("n_codebooks =",n_codebooks)
codebooks = []
for i in range(n_codebooks): 
    cb_vals = sorted(set(raw_data[:,i]))
    codebooks.append({'encode':{k: v for v, k in enumerate(cb_vals)}, 
                      'decode':{v: k for v, k in enumerate(cb_vals)}})

vocab_sizes = [len(cb['encode']) for cb in codebooks]
vocab_size = vocab_sizes[0]
vocab_size

raw_data.shape = (1115394, 1)


65

Encoder and decoder

In [6]:
encode = lambda s: [codebooks[0]['encode'][c] for c in s]
decode = lambda l: ''.join([codebooks[0]['decode'][i] for i in l]) 
test_str = 'hello!'
ilist = encode(test_str)
ret_str = decode(ilist)
assert test_str==ret_str, f"Oops. test_str={test_str}, but ret_str={ret_str}"

In [7]:
torch.manual_seed(1337)

# # here are all the unique characters that occur in this text
# chars = sorted(list(set(text)))
# vocab_size = len(chars)
# # create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long).unsqueeze(-1)  # make it like the music data
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print("train_data.shape, val_data.shape =",train_data.shape, val_data.shape)

# data loading
def get_batch(split, debug=False):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    if debug: print(f"get_batch: x.shape = {x.shape}, y.shape = {y.shape}")
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


train_data.shape, val_data.shape = torch.Size([1003854, 1]) torch.Size([111540, 1])


In [8]:
x,y = get_batch('train',debug=True)
print(f"B, T = {batch_size}, {block_size}")

get_batch: x.shape = torch.Size([16, 32, 1]), y.shape = torch.Size([16, 32, 1])
B, T = 16, 32


x is a sequence

In [9]:
x[0].T  # .T is to show it horizontal instead of vertical

tensor([[58, 53,  1, 41, 53, 56, 56, 59, 54, 58,  1, 39,  1, 51, 39, 52,  5, 57,
          1, 61, 47, 44, 43,  1, 47, 57,  0, 61, 46, 43, 52,  1]],
       device='cuda:1')

y is x shifted back by one and including new data.
in this sense only y[:,-1] is the "next token" being predicted.

In [10]:
y[0].T

tensor([[53,  1, 41, 53, 56, 56, 59, 54, 58,  1, 39,  1, 51, 39, 52,  5, 57,  1,
         61, 47, 44, 43,  1, 47, 57,  0, 61, 46, 43, 52,  1, 57]],
       device='cuda:1')

# Model definition

In [11]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_tables = nn.ModuleList([nn.Embedding(vocab_sizes[cb], n_embd) for cb in range(n_codebooks)])
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_heads = nn.ModuleList([nn.Linear(n_embd, vocab_sizes[cb]) for cb in range(n_codebooks)])


    def forward(self, idx, targets=None):
        B, T, CBS = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = 0
        for cb in range(CBS): # just sum codebook reps
            tok_emb = tok_emb + self.token_embedding_tables[cb](idx[:,:,cb]) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)

        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits_list = [self.lm_heads[cb](x) for cb in range(CBS)] # list of (B,T,vocab_sizes)

        if targets is None:
            loss = None
        else:
            lambdas = [1.0]*CBS
            targets = targets.view(B*T)
            for cb in range(CBS): 
                logits = logits_list[cb]
                B, T, C = logits.shape
                #print("logits.shape =",logits.shape,", targets.shape =",targets.shape)
                logits = logits.view(B*T, -1)
                loss = F.cross_entropy(logits, targets)

        return logits_list, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:, :]
            # get the predictions
            logits_list, loss = self(idx_cond)
            idx_next_list = []
            for cb in range(ind_cond.shape[-1]):
                # focus only on the last time step
                logits = logitslist[cb][:, -1] # becomes (B, C)
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # (B, C)
                # sample from the distribution
                idx_next_list.append(torch.multinomial(probs, num_samples=1)) # (B, 1)
            idx_next = torch.tensor(idx_next_list).to(idx.device)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



Instantiate and get ready to run

In [12]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.209729 M parameters


In [13]:
def save_checkpoint(step, model, optimizer,loss, name):
    name = name + f'_{step}.pt'
    print("Saving checkpoint to",name)
    torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, name)

def load_checkpoint(name, model, optimizer):
    checkpoint = torch.load(name)
    model.load_state_dict(checkpoint['model_state_dict'])
    m = model.to(device)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    step = checkpoint['step']
    loss = checkpoint['loss']
    return step, m, optimizer, loss

In [14]:
use_wandb = True
if use_wandb: wandb.init(project='karpathy-gpt-mini')

Do training

In [15]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if use_wandb: wandb.log(losses | {'step':iter//eval_interval})

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.4049, val loss 4.3996
step 100: train loss 2.6715, val loss 2.6830
step 200: train loss 2.5026, val loss 2.4979
step 300: train loss 2.4161, val loss 2.4296
step 400: train loss 2.3453, val loss 2.3580
step 500: train loss 2.3023, val loss 2.3212
step 600: train loss 2.2373, val loss 2.2536
step 700: train loss 2.2066, val loss 2.2192
step 800: train loss 2.1563, val loss 2.1786
step 900: train loss 2.1156, val loss 2.1487
step 1000: train loss 2.0873, val loss 2.1188
step 1100: train loss 2.0545, val loss 2.1070
step 1200: train loss 2.0220, val loss 2.0708
step 1300: train loss 2.0095, val loss 2.0497
step 1400: train loss 1.9744, val loss 2.0205
step 1500: train loss 1.9507, val loss 2.0191
step 1600: train loss 1.9310, val loss 2.0198
step 1700: train loss 1.9249, val loss 2.0059
step 1800: train loss 1.8847, val loss 1.9847
step 1900: train loss 1.8766, val loss 1.9639
step 2000: train loss 1.8625, val loss 1.9752
step 2100: train loss 1.8486, val loss 1.9539


KeyboardInterrupt: 

In [ ]:
save_checkpoint(max_iters, model, optimizer,loss, f"karpathy_base_model")

In [ ]:
if use_wandb: wandb.finish()

Generate

In [ ]:

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


In [ ]:
vocab_size